# **1. 이안류 CCTV 데이터셋**
​AI Hub의 '이안류 CCTV 데이터'는 우리나라 주요 해수욕장(해운대, 송정, 대천, 중문, 낙산)에서 이안류 발생 여부와 위치를 모니터링하기 위해 구축된 인공지능 학습용 데이터셋입니다. 해수욕장 주변에 설치된 CCTV 영상을 이미지로 변환하여, 이안류 발생 여부와 위치를 가시화하는 모델 개발에 활용할 수 있습니다. 이 데이터셋은 이안류 탐지 및 예측 시스템 개발에 필수적인 자료를 제공하며, 해수욕객의 안전을 위한 응용 서비스 구성에 활용될 수 있습니다. ​이안류는 해안에서 먼 바다로 빠르게 이동하는 폭이 좁은 바닷물의 흐름으로, 기상 상태가 양호한 경우에도 나타나며, 얕은 곳에 있던 해수욕객을 순식간에 수심이 깊은 먼 바다로 이동시켜 인명사고를 유발할 수 있습니다. 따라서 이러한 데이터셋은 해수욕장 안전 관리 및 이안류 예측 모델 개발에 중요한 역할을 합니다.

In [33]:
!unzip -q ripcurrent.zip

replace Sample/01.원천데이터/DC_MUDCH_20190705_133742.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [34]:
!pip install ultralytics opencv-python

In [35]:
import os
import random
import shutil
import cv2
import glob
import json
import yaml
import ultralytics
import matplotlib.pyplot as plt
from torchvision import transforms
from tqdm import tqdm
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

In [36]:
ultralytics.checks()

Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.3/112.6 GB disk)


In [37]:
data_root = '/content'
file_root = f'{data_root}/Sample'

train_root = f'{data_root}/train'
valid_root = f'{data_root}/valid'
test_root = f'{data_root}/test'

for folder in [train_root, valid_root, test_root]:
    if not os.path.exists(folder):
        os.makedirs(folder)
    for s in ['images', 'labels']:
        s_folder = f'{folder}/{s}'
        if not os.path.exists(s_folder):
            os.makedirs(s_folder)

In [38]:
file_list = glob.glob(f"{file_root}/02.라벨링데이터/*.json")
file_list

['/content/Sample/02.라벨링데이터/HD_GLORY_20190607_090251.json',
 '/content/Sample/02.라벨링데이터/DC_MUDCH_20190705_133806.json',
 '/content/Sample/02.라벨링데이터/SJ_WHIB1_20190617_090627.json',
 '/content/Sample/02.라벨링데이터/HD_GLORY_20190607_090249.json',
 '/content/Sample/02.라벨링데이터/SJ_WHIB1_20190617_091320.json',
 '/content/Sample/02.라벨링데이터/JM_BADA2_20190701_182824.json',
 '/content/Sample/02.라벨링데이터/NS_NSBE2_20190712_144822.json',
 '/content/Sample/02.라벨링데이터/DC_MUDCH_20190705_133731.json',
 '/content/Sample/02.라벨링데이터/JM_BADA2_20190701_182805.json',
 '/content/Sample/02.라벨링데이터/HD_PARA1_20190607_091049.json',
 '/content/Sample/02.라벨링데이터/SJ_WHIB1_20190617_090932.json',
 '/content/Sample/02.라벨링데이터/HD_GLORY_20190607_090250.json',
 '/content/Sample/02.라벨링데이터/SJ_SJHT1_20190607_090204.json',
 '/content/Sample/02.라벨링데이터/HD_GLORY_20190607_091002.json',
 '/content/Sample/02.라벨링데이터/DC_MUDCH_20190705_133730.json',
 '/content/Sample/02.라벨링데이터/SJ_SJHT1_20190607_090215.json',
 '/content/Sample/02.라벨링데이터/DC_MUDCH_201

In [39]:
def json_to_yolo_bbox(bbox, w, h):
    # print(bbox)
    x_center = ((bbox[0][0] + bbox[1][0]) / 2) / w
    y_center = ((bbox[0][1] + bbox[3][1]) / 2) / h
    width = (bbox[1][0] - bbox[0][0]) / w
    height = (bbox[3][1] - bbox[0][1]) / h

    return [x_center, y_center, width, height]

In [40]:
file = file_list[156]
print(file)

/content/Sample/02.라벨링데이터/JM_BADA1_20190701_161812.json


In [41]:
# Bounding box 형태를 YOLO 포멧으로 변환하기
result = set()
with open(file, 'r') as f:
    json_data = json.load(f)
    width, height = list(map(int, json_data["image_info"]["resolution"].split(',')))
    cls = 1

    if json_data['annotations'].get('drawing'):
        for b in json_data['annotations']['drawing']:
            yolo_bbox = json_to_yolo_bbox(b, width, height)
            # print(yolo_bbox)
            bbox_string = ' '.join([str(x) for x in yolo_bbox])
            result.add(f'{cls} {bbox_string}')

In [42]:
result = list(result)
result

[]

In [43]:
# 테스트용으로 먼저 하나만 뽑기
if result:
    with open(file.replace('json', 'txt'), 'w', encoding='utf-8') as t:
        t.write('\n'.join(result))
        print(file)

In [44]:
# 진행 게이지 표시 관련 플러그인 사용 - tqdm
for file in tqdm(file_list):
    result = set()
    with open(file, 'r') as f:
        json_data = json.load(f)
        width, height = list(map(int, json_data["image_info"]["resolution"].split(',')))
        cls = 0
        num_b = json_data["annotations"]["bounding_count"]

        if num_b > 0:
            for b in json_data["annotations"]["drawing"]:
                yolo_bbox = json_to_yolo_bbox(b, width, height)
                bbox_string = " ".join([str(x) for x in yolo_bbox])
                result.add(f"{cls} {bbox_string}")
            result = list(result)
            if result:
                with open(file.replace('json', 'txt'), 'w', encoding='utf-8') as t:
                    t.write('\n'.join(result))

100%|██████████| 360/360 [00:00<00:00, 8957.93it/s]


In [45]:
random.seed(2025)
file_list = glob.glob(f'{file_root}/02.라벨링데이터/*.txt')

random.shuffle(file_list)
test_ratio = 0.1
num_file = len(file_list)
num_file

240

In [46]:
test_list = file_list[:int(num_file*test_ratio)]
valid_list = file_list[:int(num_file*test_ratio):int(num_file*test_ratio)*2]
train_list = file_list[int(num_file*test_ratio)*2:]

In [47]:
test_list # <- 확인용

['/content/Sample/02.라벨링데이터/HD_SEAC1_20190607_092111.txt',
 '/content/Sample/02.라벨링데이터/SJ_WHIB1_20190617_091239.txt',
 '/content/Sample/02.라벨링데이터/NS_NSBE2_20190712_144936.txt',
 '/content/Sample/02.라벨링데이터/HD_SEAC1_20190607_091727.txt',
 '/content/Sample/02.라벨링데이터/HD_PARA1_20190607_091001.txt',
 '/content/Sample/02.라벨링데이터/HD_GLORY_20190607_090526.txt',
 '/content/Sample/02.라벨링데이터/HD_PARA1_20190607_090907.txt',
 '/content/Sample/02.라벨링데이터/HD_SEAC1_20190607_092107.txt',
 '/content/Sample/02.라벨링데이터/HD_GLORY_20190607_090224.txt',
 '/content/Sample/02.라벨링데이터/HD_GLORY_20190607_091003.txt',
 '/content/Sample/02.라벨링데이터/SJ_SJHT1_20190607_090357.txt',
 '/content/Sample/02.라벨링데이터/SJ_WHIB1_20190617_090930.txt',
 '/content/Sample/02.라벨링데이터/HD_GLORY_20190607_090214.txt',
 '/content/Sample/02.라벨링데이터/HD_PARA1_20190607_091053.txt',
 '/content/Sample/02.라벨링데이터/HD_PARA2_20190630_090841.txt',
 '/content/Sample/02.라벨링데이터/SJ_SJHT1_20190607_090415.txt',
 '/content/Sample/02.라벨링데이터/HD_SEAC1_20190607_091725.txt

In [56]:
# '/content/Sample/02.라벨링데이터/HD_PARA1_20190607_091002.txt'
# '/content/Sample/01.원천데이터/HD_PARA1_20190607_091002.jpg'
for i in tqdm(test_list):
    txt_name = i.split('/')[-1]
    shutil.copyfile(i, f'{test_root}/labels/{txt_name}')
    img_path = i.replace('02.라벨링데이터', '01.원천데이터').replace('txt', 'jpg')
    img_name = img_path.split('/')[-1]
    shutil.copyfile(img_path, f'{test_root}/images/{img_name}')

100%|██████████| 24/24 [00:00<00:00, 1328.50it/s]


In [57]:
for i in tqdm(valid_list):
    txt_name = i.split('/')[-1]
    shutil.copyfile(i, f'{valid_root}/labels/{txt_name}')
    img_path = i.replace('02.라벨링데이터', '01.원천데이터').replace('txt', 'jpg')
    img_name = img_path.split('/')[-1]
    shutil.copyfile(img_path, f'{valid_root}/images/{img_name}')

100%|██████████| 1/1 [00:00<00:00, 1259.17it/s]


In [58]:
for i in tqdm(train_list):
    txt_name = i.split('/')[-1]
    shutil.copyfile(i, f'{train_root}/labels/{txt_name}')
    img_path = i.replace('02.라벨링데이터', '01.원천데이터').replace('txt', 'jpg')
    img_name = img_path.split('/')[-1]
    shutil.copyfile(img_path, f'{train_root}/images/{img_name}')

100%|██████████| 192/192 [00:00<00:00, 2014.11it/s]


In [82]:
import yaml
data = dict()

data['train'] = train_root
data['valid'] = valid_root
data['test'] = test_root
data['nc'] = 1 # Corrected from 'no' to 'nc' for number of classes
data['names'] = ['yes'] # Corrected from 'name' to 'names'

with open(f'ripcurrent.yaml', 'w') as f:
    yaml.dump(data, f)

In [83]:
model = YOLO('yolov8s.pt')

results= model.train(data = 'ripcurrent.yaml', epochs = 10, batch = 8, imgsz = 224, device = 0, workers = 1, amp= False, name = 'rip_s')

Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=ripcurrent.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rip_s20, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile

RuntimeError: Dataset 'ripcurrent.yaml' error ❌ ripcurrent.yaml 'val:' key missing ❌.
'train' and 'val' are required in all data YAMLs.